In [1]:
# Install required packages
%pip install -q chromadb langchain langchain_openai langchain_community sentence-transformers python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from typing import Any, List, Mapping, Optional

In [3]:

# Import libraries
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.language_models import LLM
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import requests
import warnings

In [4]:
# Suppress warnings
warnings.filterwarnings('ignore')

In [5]:
# Load environment variables
load_dotenv()
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_model = os.getenv("OPENROUTER_MODEL", "openai/gpt-3.5-turbo")

print(f"Using OpenRouter model: {openrouter_model}")
print(f"OpenRouter API key loaded: {'Yes' if openrouter_api_key else 'No'}")

Using OpenRouter model: openai/gpt-3.5-turbo
OpenRouter API key loaded: Yes


In [6]:

class RobustOpenRouterLLM(LLM):
    """Enhanced OpenRouter LLM with better error handling"""
    
    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://openrouter.ai/api/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {openrouter_api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://localhost",
            "X-Title": "ChromaDB Demo"
        }
        data = {
            "model": openrouter_model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 2048,
            "temperature": 0.7
        }
        
        try:
            response = requests.post(url, headers=headers, json=data, timeout=30)
            response.raise_for_status()
            return response.json()['choices'][0]['message']['content']
        except requests.exceptions.RequestException as e:
            print(f"⚠️ OpenRouter API error: {str(e)}")
            return "I couldn't process your request due to an API error."

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": openrouter_model}

In [7]:
# Initialize components
llm = RobustOpenRouterLLM()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Windows\Temp\ipykernel_13236\2442575744.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [8]:
# Create sample data
data_dir = os.path.join(os.getcwd(), "data")
os.makedirs(data_dir, exist_ok=True)

In [9]:
sample_files = {
    "sample.txt": [
        "This is a sample document for testing ChromaDB with OpenRouter.",
        "ChromaDB is a vector database that can be used for semantic search.",
        "OpenRouter provides access to various LLM models through a unified API."
    ],
    "ai_models.txt": [
        "Large Language Models (LLMs) are transforming how we interact with AI.",
        "Models like GPT-3.5, GPT-4, Claude, and Llama are available through OpenRouter.",
        "Vector databases help store and retrieve embeddings for semantic search applications."
    ]
}

In [10]:

for filename, content in sample_files.items():
    filepath = os.path.join(data_dir, filename)
    if not os.path.exists(filepath):
        with open(filepath, "w") as f:
            f.write("\n".join(content))
        print(f"Created file: {filepath}")

In [11]:
# Load documents
loader = DirectoryLoader(data_dir, glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()

print(f"\nLoaded {len(documents)} documents:")
for doc in documents:
    print(f"\nDocument: {doc.metadata['source']}")
    print(f"Content: {doc.page_content[:100]}...")


Loaded 2 documents:

Document: d:\LLM_OPS\Vector_Databases\data\ai_models.txt
Content: Large Language Models (LLMs) are transforming how we interact with AI.
Models like GPT-3.5, GPT-4, C...

Document: d:\LLM_OPS\Vector_Databases\data\sample.txt
Content: This is a sample document for testing ChromaDB with OpenRouter.
ChromaDB is a vector database that c...


In [12]:
# Create ChromaDB vector store
vector_store = Chroma.from_documents(documents, embeddings)
print("\n✅ ChromaDB vector store created successfully!")


✅ ChromaDB vector store created successfully!


In [13]:
# Create QA chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer based only on the provided context."),
    ("user", "Question: {question}\nContext: {context}")
])
chain = prompt | llm | StrOutputParser()

In [14]:
def ask_question(question: str, k: int = 2):
    """Helper function to ask questions and display results"""
    try:
        print(f"\nQuestion: {question}")
        
        # Retrieve relevant documents
        docs = vector_store.similarity_search(question, k=k)
        context = "\n\n".join([doc.page_content for doc in docs])
        
        # Get answer
        answer = chain.invoke({"question": question, "context": context})
        
        print(f"\nAnswer: {answer}")
        
        print("\nSources used:")
        for i, doc in enumerate(docs):
            print(f"{i+1}. {doc.metadata['source']}")
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")

In [15]:
# Test the system
ask_question("What is ChromaDB used for?")
ask_question("What LLM models are available through OpenRouter?")
ask_question("How do vector databases help with AI applications?")


Question: What is ChromaDB used for?

Answer: ChromaDB is used for semantic search and can be integrated with OpenRouter to access various Large Language Models (LLMs) for AI interactions.

Sources used:
1. d:\LLM_OPS\Vector_Databases\data\sample.txt
2. d:\LLM_OPS\Vector_Databases\data\ai_models.txt

Question: What LLM models are available through OpenRouter?

Answer: Based on the context provided, some of the LLM models available through OpenRouter include GPT-3.5, GPT-4, Claude, and Llama.

Sources used:
1. d:\LLM_OPS\Vector_Databases\data\sample.txt
2. d:\LLM_OPS\Vector_Databases\data\ai_models.txt

Question: How do vector databases help with AI applications?

Answer: Vector databases help with AI applications by storing and retrieving embeddings for semantic search. They play a crucial role in enabling efficient and accurate search capabilities in AI systems like the LLM models available through OpenRouter.

Sources used:
1. d:\LLM_OPS\Vector_Databases\data\ai_models.txt
2. d:\LLM